In [1]:
import numpy as np
from matplotlib import pyplot as plt
import pandas as pd

import tensorflow as tf
from tensorflow.keras.models import load_model
from tensorflow_addons.optimizers import RectifiedAdam

import sys
sys.path.append('..')
sys.path.append('../Utils')
sys.path.append('../Models')

from network_tools import *
from hnet import *

%pylab inline

ModuleNotFoundError: No module named 'pandas'

# HemoNet: Model loading demonstration
The notebook shows the simple procedure by which one can load, train, and do inference with HemoNet.

## Example ECG

## Instantiate and training HemoNet
e.g. if you would like to start with our pre-trained model, then tune with your own data

In [2]:
# path to the pre-trained model
PRE_TRAINED_MODEL_PATH = '../SavedModels/ecg_supervised_224.h5'

# load it!
pre_trained_model = load_model(PRE_TRAINED_MODEL_PATH, custom_objects={
            'swish': tf.nn.swish,
            'RectifiedAdam': RectifiedAdam,
            'pearson': pearson,
        })

# take the model up to the latent layer, 224 nodes
latent = tf.keras.Model(pre_trained_model.inputs, pre_trained_model.get_layer('embed').output)

# append the additional layers to the pre-trained layers which project to the latent space
full_model = AppendNet(latent)
full_model.build((None,5000,12))

In [3]:
########## DATA ###########
'''
For the purposes of this demonstration, we generate random inputs and labels.
The actual, pressure-matched training data will be de-identified released when
possible.
'''

N = 100

X_train = np.random.rand(N,5000,12)
y_train = np.random.rand(N,4) > 0.5

In [15]:
##### TRAIN THE MODEL #####
#   as much as you want   #
#     or not at all       #

optimizer = tf.keras.optimizers.Adam() # can modify LR, of course
loss_fn = tf.keras.losses.BinaryCrossentropy()
epochs = 1

'''
Note: We wrote our own training loop so we could freely modify the procedure.
Feel free to do so (https://www.tensorflow.org/guide/keras/writing_a_training_loop_from_scratch)
'''
full_model.compile(optimizer, loss_fn)
full_model.fit(X_train, y_train, epochs=epochs)

4/4 [==============================] - 3s 313ms/step - loss: 0.8433


In [16]:
# Now, we can do inference at will

X_test = np.random.rand(100,5000,12)
y_pred = full_model.predict(X_test)

## Load the trained HemoNet
Here, we load the model that we had trained and used in our paper. All ten bootstraps are available.

Due to an issue with tensorflow related to loading operations with custom gradents, we use our own function to load the weights.

In [23]:
# first, instantiate the model

pre_trained_model = load_model(PRE_TRAINED_MODEL_PATH, custom_objects={
            'swish': tf.nn.swish,
            'RectifiedAdam': RectifiedAdam,
            'pearson': pearson,
        })

# take the model up to the latent layer, 224 nodes
latent = tf.keras.Model(pre_trained_model.inputs, pre_trained_model.get_layer('embed').output)

# append the additional layers to the pre-trained layers which project to the latent space
trained_model = AppendNet(latent)

# load the weights
trained_model = load_weights(trained_model,'../SavedModels/FinalBstraps/bs0/weights.pkl')

In [24]:
# we get an output with shape Nx4, as desired
N = 100
random_out = trained_model.predict(np.random.rand(N,5000,12))
random_out.shape

(100, 4)